<a href="https://colab.research.google.com/github/abilashkannanv/AIML/blob/main/IITG%20Project%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv('/Week 19 - Graded Mini Project - Dataset - Twitter-training.csv')
df.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [ ]:
#View Missing Values
print("Missing values before handling:\n", df.isnull().sum())

#Fill missing values
df.fillna('', inplace=True)
print("Missing values after handling:\n", df.isnull().sum())

Missing values before handling:
 2401                                                       0
Borderlands                                                0
Positive                                                   0
im getting on borderlands and i will murder you all ,    686
dtype: int64
Missing values after handling:
 2401                                                     0
Borderlands                                              0
Positive                                                 0
im getting on borderlands and i will murder you all ,    0
dtype: int64


In [ ]:
#View Duplicates
print("Number of duplicate entries before handling:", df.duplicated().sum())

# Remove duplicates
df.drop_duplicates(inplace=True)
print("Number of duplicate entries after handling:", df.duplicated().sum())



Number of duplicate entries before handling: 2700
Number of duplicate entries after handling: 0


In [ ]:
# The text cleaning function is already defined in this cell:
import re

def clean_text(text):
#     # Remove URLs
     text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
#     # Remove mentions (@username)
     text = re.sub(r'@\w+', '', text)
#     # Remove hashtags (#hashtag)
     text = re.sub(r'#\w+', '', text)
#     # Remove punctuation and special characters, keeping alphanumeric and spaces
     text = re.sub(r'[^\w\s]', '', text)
#     # Remove extra spaces
     text = re.sub(r'\s+', ' ', text).strip()
     return text


In [ ]:
# Identify the text column (assuming it's the last column after previous operations)
text_column = df.columns[-1]

print(f"Applying cleaning to column: '{text_column}'")

# Apply the cleaning function to the text column
df[text_column] = df[text_column].apply(clean_text)

# Display the first few rows of the cleaned DataFrame
print(df.head())

Applying cleaning to column: 'im getting on borderlands and i will murder you all ,'
   2401  Borderlands  Positive  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

  im getting on borderlands and i will murder you all ,  
0  I am coming to the borders and I will kill you...     
1  im getting on borderlands and i will kill you all     
2  im coming on borderlands and i will murder you...     
3  im getting on borderlands 2 and i will murder ...     
4  im getting into borderlands and i can murder y...     


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer # Import WordNetLemmatizer

nltk.download('punkt', download_dir='/tmp/nltk_data')
nltk.download('stopwords', download_dir='/tmp/nltk_data')
nltk.download('wordnet', download_dir='/tmp/nltk_data') # Download WordNet
nltk.download('omw-1.4', download_dir='/tmp/nltk_data') # Download Open Multilingual Wordnet
nltk.data.path.append('/tmp/nltk_data')

[nltk_data] Downloading package punkt to /tmp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /tmp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /tmp/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /tmp/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
stop_words = set(stopwords.words('english'))

def normalize_text(text):
    if pd.isna(text):
        return [] # Return empty list for NaN values

    # Lowercase
    text = text.lower()

    # Remove non-alphanumeric characters (keeping spaces)
    text = re.sub(r'[^a-z0-9\s]', '', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Stop-word removal
    tokens = [word for word in tokens if word not in stop_words]

    return ' '.join(tokens)

In [ ]:
# Identify the text column (assuming it's the last column)
text_column = df.columns[-1]

print(f"Applying normalization (lowercase, tokenization, stop-word removal) to column: '{text_column}'")

# Apply the normalization function to the text column
df[text_column] = df[text_column].apply(normalize_text)

# Display the first few rows of the normalized DataFrame
print(df.head())

Applying normalization (lowercase, tokenization, stop-word removal) to column: 'im getting on borderlands and i will murder you all ,'


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/tmp/nltk_data'
    - '/tmp/nltk_data'
    - '/tmp/nltk_data'
**********************************************************************


In [ ]:
import pandas as pd
import spacy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

# df is loaded and pre-processed
df = pd.read_csv('/Week 19 - Graded Mini Project - Dataset - Twitter-training.csv', encoding='utf-8')
df.fillna('', inplace=True)
df.drop_duplicates(inplace=True)

# Define the clean_text function (copied from fea84bbf)
def clean_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Identify the text column
text_column = df.columns[-1]

# Apply the cleaning function to the text column
df[text_column] = df[text_column].apply(clean_text)
# End of df pre-processing

# Load spaCy model
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])

def lemmatize_text(text):
    # Ensure text is string; if empty list, convert to empty string before processing
    if isinstance(text, list):
        text = ' '.join(text)
    if pd.isna(text) or text == '':
        return []

    doc = nlp(text.lower())

    return [
        token.lemma_
        for token in doc
        if token.is_alpha and not token.is_stop
    ]

print(f"Applying lemmatization and tokenization to column: '{text_column}'")

# Apply lemmatization
df['processed_tokens'] = df[text_column].apply(lemmatize_text)

# Display the first few rows with the new column
print(df[[text_column, 'processed_tokens']].head())

#Create padded sequences for RNN input
# Initialize Tokenizer

max_words = 10000
tokenizer = Tokenizer(num_words=max_words, oov_token="<unk>")

# Fit tokenizer on the processed tokens
tokenizer.fit_on_texts(df['processed_tokens'])

# Convert tokens to sequences of integers
sequences = tokenizer.texts_to_sequences(df['processed_tokens'])

# Pad sequences for RNN input

maxlen = max(len(s) for s in sequences) if sequences else 0
if maxlen == 0:
    # Set a default maxlen if no sequences were generated (e.g., empty df or all empty tokens)
    maxlen = 50

padded_sequences = pad_sequences(sequences, maxlen=maxlen, padding='post')

print(f"\nVocabulary size: {len(tokenizer.word_index)}")
print(f"Maximum sequence length: {maxlen}")
print(f"Shape of padded sequences: {padded_sequences.shape}")
print("First 5 padded sequences:\n", padded_sequences[:5])

Applying lemmatization and tokenization to column: 'im getting on borderlands and i will murder you all ,'
